<a href="https://colab.research.google.com/github/NoamTene/GMATcolab/blob/main/L1Station_GMAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Propagation with the GMAT API
This document walks you through the configuration and use of the GMAT API for propagation.

## Prepare the GMAT Environment
Before the API can be used, it needs to be loaded into the Python system and initialized using a GMAT startup file. This can be done from the GMAT bin folder by importing the gmatpy module, but using that approach tends to leave files in the bin folder that may annoy other users. Running from an outside folder takes a few steps, which have been captured in the load_gmat.py file in the GMAT api folder. After preparing the API for use (see the API "read me" file, api/API_README.txt), copy load_gmat.py into the folder you are using for Jupyter notebooks. Then import it:

In [ ]:
!wget https://yer.dl.sourceforge.net/project/gmat/GMAT/GMAT-R2022a/gmat-ubuntu-x64-R2022a.tar.gz
!tar -xf gmat-ubuntu-x64-R2022a.tar.gz
!python "GMAT/R2022a/api/BuildApiStartupFile.py"
!sed -i 's/<TopLevelGMATFolder>/\/content\/GMAT\/R2022a/' GMAT/R2022a/api/load_gmat.py
%cd GMAT/R2022a/api

--2023-08-27 00:28:32--  https://yer.dl.sourceforge.net/project/gmat/GMAT/GMAT-R2022a/gmat-ubuntu-x64-R2022a.tar.gz
Resolving yer.dl.sourceforge.net (yer.dl.sourceforge.net)... 94.20.154.18, 2a01:6480:20::5
Connecting to yer.dl.sourceforge.net (yer.dl.sourceforge.net)|94.20.154.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 390518023 (372M) [application/x-gzip]
Saving to: ‘gmat-ubuntu-x64-R2022a.tar.gz.1’

gmat-ubuntu-x64-R20 100%[===================>] 372.43M  13.7MB/s    in 29s     

2023-08-27 00:29:02 (12.7 MB/s) - ‘gmat-ubuntu-x64-R2022a.tar.gz.1’ saved [390518023/390518023]

/content/GMAT/R2022a/api


In [ ]:
from load_gmat import *
import matplotlib.pyplot as plt

## Configure a Spacecraft
We'll need a spacecraft to propagate.  The following lines provide a basic Spacecraft configuration:

In [ ]:
EML1 = gmat.Construct("LibrationPoint","EML1")
EML1.SetField("OrbitColor","GreenYellow");
EML1.SetField("TargetColor","DarkGray");
EML1.SetField("Primary","Earth");
EML1.SetField("Secondary","Luna");
EML1.SetField("Point","L1");

In [ ]:
EM_L1S = gmat.Construct("Spacecraft","EM_L1S")
EM_L1S.SetField("DateFormat", "UTCGregorian")
EM_L1S.SetField("Epoch", "01 Jan 2025 00:00:00.000")
EM_L1S.SetField("CoordinateSystem", "EarthMoonL1CS")
EM_L1S.SetField("DisplayStateType", "Cartesian")
EM_L1S.SetField("X", 0)
EM_L1S.SetField("Y", 0)
EM_L1S.SetField("Z", 0)
EM_L1S.SetField("VX", 0)
EM_L1S.SetField("VY", 0)
EM_L1S.SetField("VZ", 0)
print(EM_L1S.GetGeneratingString(0))


Create Spacecraft EM_L1S;
GMAT EM_L1S.DateFormat = UTCGregorian;
GMAT EM_L1S.Epoch = '01 Jan 2025 00:00:00.000';
GMAT EM_L1S.CoordinateSystem = EarthMoonL1CS;
GMAT EM_L1S.DisplayStateType = Cartesian;
GMAT EM_L1S.X = 0;
GMAT EM_L1S.Y = 0;
GMAT EM_L1S.Z = 0;
GMAT EM_L1S.VX = 0;
GMAT EM_L1S.VY = 0;
GMAT EM_L1S.VZ = 0;
GMAT EM_L1S.DryMass = 850;
GMAT EM_L1S.Cd = 2.2;
GMAT EM_L1S.Cr = 1.8;
GMAT EM_L1S.DragArea = 15;
GMAT EM_L1S.SRPArea = 1;
GMAT EM_L1S.SPADDragScaleFactor = 1;
GMAT EM_L1S.SPADSRPScaleFactor = 1;
GMAT EM_L1S.AtmosDensityScaleFactor = 1;
GMAT EM_L1S.ExtendedMassPropertiesModel = 'None';
GMAT EM_L1S.NAIFId = -10000001;
GMAT EM_L1S.NAIFIdReferenceFrame = -9000001;
GMAT EM_L1S.OrbitColor = Red;
GMAT EM_L1S.TargetColor = Teal;
GMAT EM_L1S.OrbitErrorCovariance = [ 1e+70 0 0 0 0 0 ; 0 1e+70 0 0 0 0 ; 0 0 1e+70 0 0 0 ; 0 0 0 1e+70 0 0 ; 0 0 0 0 1e+70 0 ; 0 0 0 0 0 1e+70 ];
GMAT EM_L1S.CdSigma = 1e+70;
GMAT EM_L1S.CrSigma = 1e+70;
GMAT EM_L1S.Id = 'SatId';
GMAT EM_L1S.Attitude = Coo

In [ ]:
#sat = gmat.Construct("Spacecraft","LeoSat")
#sat.SetField("DryMass", 50)
#sat.SetField("Cd", 2.2)
#sat.SetField("Cr", 1.8)
#sat.SetField("DragArea", 1.5)
#sat.SetField("SRPArea", 1.2)

## Configure the Forces
Next we'll set up a force model.  For this example, we'll use an Earth 8x8 potential model, with Sun and Moon point masses and Jacchia-Roberts drag.  In GMAT, forces are collected in the ODEModel class.  That class is scripted as a "ForceModel" in the script language.  The API accepts either.  The force model is built and its (empty) contents displayed using

In [ ]:
fm = gmat.Construct("ForceModel", "FM")

earthgrav = gmat.Construct("GravityField")
earthgrav.SetField("BodyName","Earth")
earthgrav.SetField("Degree",4)
earthgrav.SetField("Order",4)
earthgrav.SetField("PotentialFile","JGM2.cof")
fm.AddForce(earthgrav)

moongrav = gmat.Construct("PointMassForce")
moongrav.SetField("BodyName","Luna")
fm.AddForce(moongrav)

sungrav = gmat.Construct("PointMassForce")
sungrav.SetField("BodyName","Sun")
fm.AddForce(sungrav)

# Drag using Jacchia-Roberts
#jrdrag = gmat.Construct("DragForce")
#jrdrag.SetField("AtmosphereModel","JacchiaRoberts")
#atmos = gmat.Construct("JacchiaRoberts")
#jrdrag.SetReference(atmos)
#fm.AddForce(jrdrag)

fm.Help()


ForceModel  FM

   Field                                   Type   Value
   --------------------------------------------------------

   CentralBody                           Object   Earth
   PrimaryBodies                    ObjectArray   {Earth}
   PolyhedralBodies                 ObjectArray   {}
   PointMasses                      ObjectArray   {Luna, Sun}
   Drag                                  Object   None
   SRP                                    OnOff   Off
   RelativisticCorrection                 OnOff   Off
   ErrorControl                            List   RSSStep
   UserDefined                      ObjectArray   {}



''

In GMAT, the force model scripting shows the settings for each force.  In the API, you can examine the settings for the individual forces:

or, with a little work, the GMAT scripting for the complete force model:

In [ ]:
print(fm.GetGeneratingString(0))

Create ForceModel FM;
GMAT FM.CentralBody = Earth;
GMAT FM.PrimaryBodies = {Earth};
GMAT FM.PointMasses = {Luna, Sun};
GMAT FM.Drag = None;
GMAT FM.SRP = Off;
GMAT FM.RelativisticCorrection = Off;
GMAT FM.ErrorControl = RSSStep;
GMAT FM.GravityField.Earth.Degree = 4;
GMAT FM.GravityField.Earth.Order = 4;
GMAT FM.GravityField.Earth.StmLimit = 100;
GMAT FM.GravityField.Earth.PotentialFile = 'JGM2.cof';
GMAT FM.GravityField.Earth.TideModel = 'None';



## Configure the Integrator
Finally, in order to propagate, we need an integrator.  For this example, we'll use a Prince-Dormand 7(8) Runge-Kutta integrator.  The propagator is set using the code

In [ ]:
pdprop = gmat.Construct("Propagator","PDProp")
gator = gmat.Construct("PrinceDormand78", "Gator")
pdprop.SetReference(gator)
pdprop.SetField("InitialStepSize", 60.0)
pdprop.SetField("Accuracy", 1.0e-13)
pdprop.SetField("MinStep", 0.0)
pdprop.SetReference(fm);

It also need to know the object that is propagated.  For this example, that is the spacecraft constructed above:

In [ ]:
pdprop.AddPropObject(EM_L1S)
print(pdprop.GetGeneratingString(0)

Create ForceModel FM;
GMAT FM.CentralBody = Earth;
GMAT FM.PrimaryBodies = {Earth};
GMAT FM.PointMasses = {Luna, Sun};
GMAT FM.Drag = None;
GMAT FM.SRP = Off;
GMAT FM.RelativisticCorrection = Off;
GMAT FM.ErrorControl = RSSStep;
GMAT FM.GravityField.Earth.Degree = 4;
GMAT FM.GravityField.Earth.Order = 4;
GMAT FM.GravityField.Earth.StmLimit = 100;
GMAT FM.GravityField.Earth.PotentialFile = 'JGM2.cof';
GMAT FM.GravityField.Earth.TideModel = 'None';

Create Propagator PDProp;
GMAT PDProp.FM = FM;
GMAT PDProp.Type = PrinceDormand78;
GMAT PDProp.InitialStepSize = 60;
GMAT PDProp.Accuracy = 1e-13;
GMAT PDProp.MinStep = 0;
GMAT PDProp.MaxStep = 2700;
GMAT PDProp.MaxStepAttempts = 50;
GMAT PDProp.StopIfAccuracyIsViolated = true;



## Initialize the System and Propagate a Step
Finally, the system can be initialized and fired to see a single propagation step.  Some of the code displayed here will be folded into the API's Initialize() function.  For now, the steps needed to initialize the system for a propagation step are:

# New Section

In [ ]:
# Perform top level initialization
gmat.Initialize()
# Perform the integation subsysem initialization
pdprop.PrepareInternals()

# Refresh the integrator reference
gator = pdprop.GetPropagator()

and we can then propagate, and start accumulating the data

In [ ]:
# Take a 60 second step, showing the state before and after, and start buffering
# Buffers for the data
time = []
pos = []
vel = []

gatorstate = gator.GetState()
t = 0.0
r = []
v = []
for j in range(3):
    r.append(gatorstate[j])
    v.append(gatorstate[j+3])
time.append(t)
pos.append(r)
vel.append(v)

print("Starting state:\n", t, r, v)

# Take a step and buffer it
gator.Step(60.0)
gatorstate = gator.GetState()
t = t + 60.0
r = []
v = []
for j in range(3):
    r.append(gatorstate[j])
    v.append(gatorstate[j+3])
time.append(t)
pos.append(r)
vel.append(v)

print("Propped state:\n", t, r, v)

Starting state:
 0.0 [0.0, 0.0, 0.0] [0.0, 0.0, 0.0]


APIException: ignored

Finally, we can run for a few orbits and show the results

In [ ]:
for i in range(360):
    # Take a step and buffer it
    gator.Step(60.0)
    gatorstate = gator.GetState()
    t = t + 60.0
    r = []
    v = []
    for j in range(3):
        r.append(gatorstate[j])
        v.append(gatorstate[j+3])
    time.append(t)
    pos.append(r)
    vel.append(v)

plt.rcParams['figure.figsize'] = (15, 5)
positions = plt.plot(time, pos)

The velocities can also be plotted:

In [ ]:
velocities = plt.plot(time, vel)